# Intro to Data Science  
## Part V - Text Mining  

### Table of Contents  
- #### Text Mining  
    - <a href="#what-is-text-mining">Theory</a>  
    - <a href="#text-mining-in-practice">In practice</a>  
    - <a href="#bag-of-words-representation">Vectorizing documents</a>  
    - <a href="#tf-idf">Normalizing document vectors</a>  
    - <a href="#hashing">Vectorizing large corpora</a>  
    - <a href="#latent-semantic-indexing">Topic modeling</a>  
    - <a href="#document-similarity-metrics">Document similarity</a>  
- #### Artificial Neural Networks (ANN)  
    - <a href="#neural-networks">Single-layer networks</a>  
    - <a href="#solving-non-linear-problems">Multi-layer networks</a>  

---

## What is Text Mining?  
Text mining (or text analytics) is the process of extracting structured, meaningful features from natural language texts. Since most raw text data is unstructured, we need **Natural Language Processing (NLP)** techniques, statistical modeling, and machine learning to transform it into a form suitable for analysis.  

## Why is Text Mining Important?  
Around **80% of generated data** is unstructured, meaning it doesn’t fit neatly into tables or databases. This includes:  

- Emails, meeting notes, reports  
- Social media posts, chat logs  
- Articles, books, and research papers  
- Transcripts of voice recordings and videos  

While these data sources may contain metadata (e.g., length, topic, or category), extracting deeper insights requires transforming them into structured formats. For example, voice recordings and videos can be **transcribed into text**, which can then be processed just like any other document.  

### Common Use Cases  
Text mining enables a wide range of applications, including:  

- **Document similarity analysis** (e.g., finding related articles)  
- **Deduplication** (e.g., identifying near-duplicate documents)  
- **Document clustering** (e.g., grouping news articles by topic)  
- **Topic extraction** (e.g., summarizing themes in a collection of documents)  
- **Sentiment analysis** (e.g., determining if reviews are positive or negative)  
- **Automated annotation** (e.g., adding tags to documents based on content)  
- **Text classification** (e.g., spam detection in emails)  
- **Text filtering** (e.g., removing offensive content)  

---

## Tools for Text Mining  
### NLP Techniques  
- **Tokenization** – Splitting text into words or phrases  
- **Stemming & Lemmatization** – Reducing words to their base form (e.g., "running" → "run")  
- **Part-of-Speech (POS) Tagging** – Identifying grammatical roles (noun, verb, adjective, etc.)  
- **Stopword Filtering** – Removing common but uninformative words (e.g., "the", "and", "is")  
- **Bag-of-Words (BoW) Representation** – Converting text into a numerical format  
- **TF-IDF Transformation** – Adjusting term importance based on document frequency  

### Other Key Tools  
- **Word Embeddings (e.g., Word2Vec, GloVe)** – Capturing word meanings in vector form  
- **Hashing** – Efficient vectorization of large datasets  
- **Similarity Metrics** – Measuring how alike documents are (e.g., cosine, Jaccard, Levenshtein)  
- **Matrix Factorization** – Reducing dimensionality for better topic modeling  

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import scipy.sparse as sp
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Text Mining in Practice  

### 1. Reading and Examining the Data  

Before diving into text processing, we need to understand the raw data.  
The collection of text documents we analyze is called a **corpus**.  

In [ ]:
with open('./data/SMSSpamCollection', 'rb') as spamfile:
    corpus = [line.decode('utf-8').strip() for line in spamfile]
len(corpus)

In [ ]:
for text in corpus[:5]:
    print(text)

We can see that the data is in TSV format, read it accordingly.

In [ ]:
corpus = pd.read_csv('./data/SMSSpamCollection', sep='\t', names=['label', 'message'])

In [ ]:
corpus.groupby('label').describe()

In [ ]:
corpus['length'] = corpus.message.str.len()
corpus.head()

In [ ]:
corpus['wordcount'] = corpus.message.str.split().str.len()
corpus.head()

In [ ]:
corpus.length.plot(bins=20, kind='hist');

In [ ]:
corpus.length.describe()

910 long sms???

In [ ]:
corpus.loc[corpus.length > 900, 'message'].values

Is there a difference between spam and ham messages?

In [ ]:
corpus[['length', 'label']].hist(bins=50, by='label', sharex=True);

Why not try a simple predictor?

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
splitted = train_test_split(corpus.length.values[:, np.newaxis], # we need a matrix, not a vector
                            corpus.label.values,
                            test_size=.25,
                            stratify=corpus.label.values,
                            random_state=42)
X_train, X_test, y_train, y_test = splitted

In [ ]:
pipe = Pipeline([('nb', MultinomialNB())])
pipe.fit(X_train, y_train)

In [ ]:
accuracy_score(y_test, pipe.predict(X_test))

Our baseline accuracy is around 87%. Not bad, but we haven’t done any preprocessing yet. Let’s improve this!

### 2. Preprocessing
#### a) [Bag-of-words representation](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)

The **Bag-of-Words (BoW) model** converts text documents into numerical vectors by counting word occurrences. Each unique word gets a fixed position in a vector, and the corresponding value represents how often it appears in the document.

Example:  
Let’s say we have the following documents:
```python
docs = ["I like trains.", "Trains are like big cars.", "I like big cars"]
```
The vocabulary (set of unique words) would be:
```python
features = {'I': 0, 'like': 1, 'trains': 2, 'are': 3, 'big': 4, 'cars': 5}
```
And the vectorized form of the documents would be:
```python
vectors = [[1, 1, 1, 0, 0, 0],  # "I like trains."
           [0, 1, 1, 1, 1, 1],  # "Trains are like big cars."
           [1, 1, 0, 0, 1, 1]]  # "I like big cars."
```
Each row corresponds to a document, and each column represents a word. The numbers indicate word frequency.

Fortunately, we don’t need to build this from scratch — `scikit-learn` provides a built-in [`CountVectorizer`](http://scikit-learn.org/stable/modules/feature_extraction.html#the-bag-of-words-representation) for converting text into a bag-of-words representation.

Let's try out our little example:

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cntvec = CountVectorizer()
docs = ["I like trains.",
        "Trains are like big cars.",
        "I like big cars"]

cntvec.fit_transform(docs).todense(), cntvec.vocabulary_

#### N-grams

N-grams are continuous sequences of $n$ words from a given text. They help capture contextual information that individual words might miss.  

For example, a **2-gram (bigram)** representation of the sentence `"I like trains."` would be:

```python
[("I", "like"), ("like", "trains")]
```

By increasing $n$, we can capture more context. A **3-gram (trigram)** for the same sentence would be:

```python
[("I", "like", "trains")]
```

N-grams are especially useful for tasks like **text classification**, **speech recognition**, and **predictive text**.

In [ ]:
cntvec = CountVectorizer(ngram_range=(2, 2))
cntvec.fit_transform(docs).todense(), cntvec.vocabulary_

#### Minimum and Maximum Document Frequency

The parameters **`min_df`** and **`max_df`** in **TF-IDF vectorization** help control which terms are included in the vocabulary by setting frequency thresholds:  

- **`min_df`** (minimum document frequency): Excludes rare words that appear in fewer than the specified number (or percentage) of documents.  
- **`max_df`** (maximum document frequency): Removes very common words that appear in more than the specified number (or percentage) of documents.  

This filtering step helps **reduce noise and improve model performance** by focusing on informative terms while ignoring overly rare or common words.

In [ ]:
cntvec = CountVectorizer(max_df=1)
cntvec.fit_transform(docs).todense(), cntvec.vocabulary_

In [ ]:
cntvec = CountVectorizer(min_df=3)
cntvec.fit_transform(docs).todense(), cntvec.vocabulary_

### Advanced Tokenization  

When building a vocabulary, words are analyzed and transformed to reduce redundancy and improve text representation.  
By default, Scikit-Learn's tokenizer **lowercases words** and **removes short and stop words**, but it does not apply deeper transformations.  

However, more advanced **Natural Language Processing (NLP) techniques** can help extract meaningful base words.  

### Lemmatization  

**Lemmatization** reduces words to their **dictionary root form** (a valid word you would find in a dictionary).  

For example:  
- `"are"` → `"be"`  
- `"trains"` → `"train"`  
- `"running"` → `"run"`  
- `"better"` → `"good"`  

Lemmatization uses linguistic rules and considers context, making it more accurate than simple stemming.  

### Stemming  

**Stemming** is a simpler technique that removes affixes (prefixes/suffixes) from words to obtain the root form.  
Unlike lemmatization, it **does not** consider context or grammar—it just chops off endings.  

Example:  
- `"running"` → `"run"`  
- `"flies"` → `"fli"`  
- `"happily"` → `"happili"`  
- `"better"` → `"better"` (incorrectly unchanged)  

Since stemming is based on crude rules, it sometimes produces non-existent words, but it can still be useful for reducing vocabulary size in search engines or text classification tasks.  

### Using `nltk` for Stemming  

A popular NLP library for stemming and other text-processing tasks is [`Natural Language Toolkit (nltk)`](https://www.nltk.org/).  

#### Installation  
```bash
conda activate szisz_ds_2025
pip install nltk
```

In [ ]:
from nltk.stem import PorterStemmer

In [ ]:
stemmer = PorterStemmer()
stemmer.stem('trains')

#### Lemmatization

Lemmatization is similar to stemming, but with an important difference: **Lemmatization always returns real words**, whereas stemming might produce non-existent ones.

Lemmatization uses linguistic knowledge (e.g., dictionaries and grammar rules) to find a word’s root form, making it **more accurate than stemming** but also **slower**.  

In [ ]:
import nltk
nltk.download('wordnet')

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize('are', pos='v')

Lemmatization provides better accuracy but is computationally more expensive than stemming.  

We can use **lemmatization** to create **custom text analyzers** in `CountVectorizer`, allowing us to process text more effectively before vectorization!

In [ ]:
def split_into_lemmas(message):
    message = ''.join([char for char in message.lower()
                       if char.isalnum() or char.isspace()])
    return [lemmatizer.lemmatize(word, pos='v') 
            for word in message.split()]

[split_into_lemmas(doc) for doc in docs]

##### TextBlob: A User-Friendly NLP Library  

[`TextBlob`](https://textblob.readthedocs.io/en/dev/) is a simple yet powerful NLP library that makes text processing easy.  
It provides an intuitive interface for common NLP tasks, including **lemmatization**, **sentiment analysis**, and **POS tagging**.  

**Why use TextBlob?**
- Great for quick and easy text analysis.
- Requires minimal setup.
- Good for beginners in NLP.

**Installation**:
```bash
conda activate szisz_df_2025
pip install textblob
python -m textblob.download_corpora
```

In [ ]:
from textblob import TextBlob

In [ ]:
def split_into_lemmas(message):
    message = message.lower()
    words = TextBlob(message).words
    return [word.lemma for word in words]

[split_into_lemmas(doc) for doc in docs]

In [ ]:
cntvec = CountVectorizer(analyzer=split_into_lemmas)
cntvec.fit_transform(docs).todense(), cntvec.vocabulary_

Let's insert our vectorizer to our pipeline!

In [ ]:
splitted = train_test_split(corpus.message,
                            corpus.label.values,
                            test_size=.25,
                            stratify=corpus.label.values,
                            random_state=42)
X_train, X_test, y_train, y_test = splitted

In [ ]:
pipe = Pipeline([
    ('cntvec', CountVectorizer(analyzer=split_into_lemmas, min_df=10, max_df=.5)),
    ('nb', MultinomialNB())
])

pipe.fit(X_train, y_train)
accuracy_score(y_test, pipe.predict(X_test))

In [ ]:
len(pipe['cntvec'].vocabulary_)

##### SpaCy: A High-Performance NLP Library

[`spacy`](https://spacy.io/) is a **more advanced** NLP library designed for **speed and accuracy**.  
It includes powerful tokenization, lemmatization, named entity recognition, and dependency parsing.

**Why use SpaCy?**
- **Faster and more efficient** than NLTK and TextBlob.
- Supports **pre-trained language models** for various NLP tasks.
- Ideal for **large-scale** NLP applications.

**Installation** (*requires admin rights!*):
```bash
conda activate szisz_ds_2025
pip install spacy
python -m spacy download en_core_web_sm
```

In [ ]:
import spacy

In [ ]:
nlp = spacy.load('en_core_web_sm')
[token.lemma_ for token in nlp(docs[0])]

In [ ]:
pd.DataFrame([
    {'text': token.text, 
     'lemma': token.lemma_, 
     'POS': token.pos_, 
     'tag': token.tag_, 
     'dep': token.dep_,
     'shape': token.shape_,
     'is_alpha': token.is_alpha, 
     'is_stop': token.is_stop}
    for token in nlp(docs[0])
]).set_index('text').transpose()

In [ ]:
pipe = Pipeline([
    ('cntvec', CountVectorizer(analyzer=lambda x: [w.lemma_ for w in nlp(x)], min_df=10, max_df=.5)),
    ('nb', MultinomialNB())
])
pipe.fit(X_train, y_train)
accuracy_score(y_test, pipe.predict(X_test))

#### b) [TF-IDF](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)  

TF-IDF (Term Frequency - Inverse Document Frequency) is a method for **normalizing word counts** in a document. It helps identify words that are important **within a document** but not overly common **across all documents** in a corpus.  

TF-IDF is the product of two components:  

1. **Term Frequency (TF)** – Measures how often a word appears in a document.  
   $$
   \mathrm{tf} (t,d) = \frac{1}{2} + \frac{f_{t,d}}{2 \cdot \max\{f_{t',d} : t' \in d\}}
   $$  
   where:  
   - $( f_{t,d} )$ is the count of term $( t )$ in document $( d )$.  
   - The denominator normalizes by the highest term frequency in ( $d$ ) to avoid bias toward longer documents.  

2️. **Inverse Document Frequency (IDF)** – Measures how **rare** a word is across documents.  
   $$
   \mathrm{idf}(t, D) = \log \frac{N}{|\{d \in D: t \in d\}|}
   $$  
   where:  
   - $( N )$ is the total number of documents in the corpus.  
   - $( |\{d \in D: t \in d\}| )$ counts how many documents contain the term $( t )$.  
   - Words that appear in **many** documents get a **low** IDF score, while rare words get a **high** IDF score.  

**Why use TF-IDF?**  
- **Better than raw word counts** – avoids favoring common words like "the" or "is."  
- **Useful for keyword extraction** – identifies important terms in a document.  
- **A foundation for search engines** – helps rank relevant documents based on query terms.  

In `scikit-learn`, we can compute TF-IDF using [`TfidfVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

In [ ]:
pipe = Pipeline([
    ('cntvec', CountVectorizer(analyzer=split_into_lemmas, min_df=5, max_df=.9)),
    ('tfidf', TfidfTransformer()),
    ('nb', MultinomialNB())
])
pipe.fit(X_train, y_train)
accuracy_score(y_test, pipe.predict(X_test))

In [ ]:
np.argsort([5, 3, 7, 9, 1])

In [ ]:
for word in np.argsort(pipe['tfidf'].idf_)[-20:][::-1]:
    print(word, pipe['cntvec'].get_feature_names_out()[word], pipe['tfidf'].idf_[word])

#### c) [Hashing](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html)  

When working with **very large text corpora**, a major challenge is **memory usage**. As the number of documents grows, the vocabulary size increases, requiring more memory to store word indices.  

To overcome this, we use the [**hashing trick**]((http://scikit-learn.org/stable/modules/feature_extraction.html#feature-hashing)), which replaces explicit vocabulary storage with a **fixed-size feature space**. Instead of keeping a dictionary of words, we apply a **hash function** that maps words directly to feature indices. This allows **constant memory usage**, regardless of corpus size!  

**How does it work?**  
- Each word is processed by a **hash function** that outputs an integer index.  
- The hashed index determines **where** the word contributes in the feature matrix.  
- Since hash functions can produce **collisions** (different words mapping to the same index), the method works best with **high-dimensional spaces** to minimize information loss.  

**Advantages:**  
- **Memory-efficient** – No need to store a growing vocabulary.  
- **Fast transformation** – Works well for streaming or real-time applications.  
- **Scalable** – Handles massive datasets without increasing memory footprint.  

**Disadvantages:**  
- **No inverse transform** – Once words are hashed, we **lose interpretability** (i.e., we can’t recover the original words).  
- **Possible collisions** – Different words may be mapped to the same index, causing minor accuracy loss.  

**In `scikit-learn`,** we can use [`HashingVectorizer`](http://scikit-learn.org/stable/modules/feature_extraction.html#feature-hashing) to efficiently transform text data without storing a vocabulary. Let’s try it in action!  

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

In [ ]:
pipe = Pipeline([
    ('hash', HashingVectorizer(analyzer=split_into_lemmas, n_features=1000, alternate_sign=False)),
    ('nb', MultinomialNB())
])
pipe.fit(X_train, y_train)
accuracy_score(y_test, pipe.predict(X_test))

### 3. Latent Semantic Indexing (LSI)  

_"Latent Semantic Analysis (LSA) is a technique in natural language processing that analyzes the relationships between a set of documents and the terms they contain by identifying underlying **concepts**. The key idea is that words with similar meanings tend to appear in similar contexts."_ — [Wikipedia](https://en.wikipedia.org/wiki/Latent_semantic_analysis)  

#### How Does It Work?  
LSA is based on the **distributional hypothesis**, which states that words appearing in similar contexts tend to have similar meanings. To uncover these **hidden structures in text**, we apply **Singular Value Decomposition (SVD)** to a **Tf-Idf matrix** of the corpus.  

Mathematically, given a **term-document matrix** $ A $ (where rows are words, columns are documents, and values are Tf-Idf scores), we apply **SVD**:  

$$
A = U \Sigma V^T
$$

where:  
- $ U $ contains **word-topic associations**  
- $ \Sigma $ contains **importance weights** of topics  
- $ V^T $ contains **document-topic associations**  

By keeping only the **top $ k $ singular values**, we reduce noise and capture the **most important latent topics** in the dataset.  

#### Why Use LSA?  
- **Reduces noise** – Helps remove irrelevant word variations (e.g., synonyms).  
- **Captures hidden relationships** – Groups words and documents by meaning, not just surface-level similarity.  
- **Improves document retrieval** – Useful in search engines and recommendation systems.  

#### Limitations  
- **Computationally expensive** – Requires matrix decomposition, which is slower than simpler methods.  
- **Fixed topics** – Unlike more advanced methods (e.g., LDA), LSA does not model **topic probabilities**, only a **fixed representation**.  

In `scikit-learn`, LSA can be implemented using [`TruncatedSVD`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html), which efficiently performs dimensionality reduction. Let’s see it in action!  

In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC

In [ ]:
pipe = Pipeline([
    ('cntvec', CountVectorizer(analyzer=split_into_lemmas)),
    ('tfidf', TfidfTransformer(sublinear_tf=True)),
    ('svd', TruncatedSVD(n_components=300, random_state=42)),
    ('svm', SVC(C=300))
])
pipe.fit(X_train, y_train)
accuracy_score(y_test, pipe.predict(X_test))

In [ ]:
feat_names = pipe['cntvec'].get_feature_names_out()
topics = pipe['svd'].components_
topic_str = pipe['svd'].explained_variance_ratio_

In [ ]:
def get_most_important(topic, feat_names):
    indeces = np.argsort(topic)[::-1]
    terms = [feat_names[weightIndex] for weightIndex in indeces[:10]]    
    weights = [topic[weightIndex] for weightIndex in indeces[:10]] 
    return dict(zip(terms, weights))

In [ ]:
for i in range(10):
    print(i, topic_str[i], get_most_important(topics[i], feat_names))
    print('-' * 80)

### 4. Document Similarity Metrics  

When comparing documents, **Euclidean distance** is often not ideal. Since text data is high-dimensional and sparse, measuring similarity using raw distances can be misleading. Instead, we use **cosine similarity**, which measures the **angle** between two document vectors rather than their absolute distance.  

#### Cosine Similarity Formula  

Given two document vectors $ \mathbf{A} $ and $ \mathbf{B} $, the cosine similarity is defined as:

$$
\text{cosine similarity}(\mathbf{A}, \mathbf{B}) = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \|\mathbf{B}\|}
$$

where:  
- $ \mathbf{A} \cdot \mathbf{B} $ is the **dot product** of the two vectors  
- $ \|\mathbf{A}\| $ and $ \|\mathbf{B}\| $ are the **Euclidean norms** (magnitudes) of the vectors  

#### Why Cosine Similarity?  
- **Insensitive to document length** – Longer documents won’t automatically seem more dissimilar.  
- **Captures semantic similarity** – Focuses on relative word usage rather than absolute frequencies.  
- **Efficient to compute** – Especially with sparse matrix optimizations.  

Since **Tf-Idf vectors** already capture word importance, cosine similarity can be computed simply by taking the **dot product** of two document vectors. This makes it a natural choice for **document retrieval**, **clustering**, and **classification** tasks.  

Let’s see how to implement it in Python using `scikit-learn`!  

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def split_to_lemmas_and_filter(message):
    lemmas = split_into_lemmas(message)
    return [lemma for lemma in lemmas 
            if lemma not in ENGLISH_STOP_WORDS]
    
tfidf = TfidfVectorizer(analyzer=split_to_lemmas_and_filter,
                        min_df=10,
                        max_df=.5).fit(corpus.message)
vects = tfidf.transform(corpus.message)

In [ ]:
vect = tfidf.transform([corpus.message[0]])
corpus.message[0], vect

In [ ]:
sims = vects.dot(vect.T).toarray().flatten()
most_similar = np.argsort(sims)[-10:][::-1]

for i, index in enumerate(most_similar):
    print(i, sims[index])
    print(corpus.message[index])
    print('-' * 80)

### 5. Named Entity Recognition (NER)

Named Entity Recognition (NER) is a key **Natural Language Processing (NLP)** technique that identifies and categorizes important entities in text, such as **names, locations, dates, organizations, and more**. It helps extract structured information from unstructured text.  

For example, in the sentence:  
*"Apple Inc. is headquartered in Cupertino, California, and was founded by Steve Jobs."*  

A NER system would identify:  
- **Apple Inc.** → *Organization*  
- **Cupertino, California** → *Location*  
- **Steve Jobs** → *Person*  

#### Why is NER useful?
NER is widely used for:  
- **Information extraction** (e.g., extracting company names from financial reports).  
- **Question answering** (e.g., recognizing dates, locations, and names in queries).  
- **Content classification** (e.g., tagging people, places, and organizations in news articles).  

#### Tools for NER
There are several NLP libraries that provide pre-trained NER models:  
- **spaCy** – A fast and efficient NLP library with built-in NER models.  
- **NLTK** – Includes simple NER tools but requires additional training.  
- **TextBlob** – Provides a simpler interface for NER tasks.  
- **Transformers (Hugging Face)** – State-of-the-art models for entity recognition.  

Let's see how to use **spaCy** for Named Entity Recognition!

In [ ]:
# Example: Apply NER on a sample text from the dataset
sample_text = X_train.iloc[5]  # Take the first training sample
doc = nlp(sample_text)

# Print detected entities
print("Original Text:", sample_text)
print("\nNamed Entities:")
for ent in doc.ents:
    print(f"{ent.text} → {ent.label_}")

# Visualizing NER results (Jupyter Notebook only)
spacy.displacy.render(doc, style="ent", jupyter=True)

### 6. Sentiment Analysis

Sentiment analysis is the process of determining whether a piece of text conveys a **positive**, **negative**, or **neutral** sentiment. It is widely used in **social media monitoring**, **customer feedback analysis**, and **brand reputation management**.

There are two main approaches to sentiment analysis:
1. **Lexicon-based methods**: Use predefined dictionaries of words with sentiment scores (e.g., VADER, TextBlob).
2. **Machine learning-based methods**: Train a classifier (e.g., logistic regression, neural networks) on labeled sentiment data.

In this section, we will demonstrate **sentiment analysis** using the `TextBlob` library, which provides an easy-to-use interface for extracting sentiment polarity from text.

In [ ]:
# Example function for sentiment analysis
def analyze_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity  # Returns a value between -1 (negative) and 1 (positive)

# Apply sentiment analysis to the dataset
X_train_sentiment = X_train.apply(analyze_sentiment)
X_test_sentiment = X_test.apply(analyze_sentiment)

# Quick look at results
sample_texts = X_test[:5]
sample_sentiments = X_test_sentiment[:5]

for text, sentiment in zip(sample_texts, sample_sentiments):
    sentiment_label = "Positive" if sentiment > 0 else "Negative" if sentiment < 0 else "Neutral"
    print(f"Text: {text}\nSentiment Score: {sentiment:.2f} ({sentiment_label})\n")

## Model of the Week:
### Neural Networks

<img src="pics/artificial_neuron.png" width="300" align="left"/>

Artificial Neural Networks (ANNs) are a supervised machine learning method used for both classification and regression tasks. Inspired by the structure and functioning of the human brain, ANNs consist of basic computational units called [**neuron**](https://en.wikipedia.org/wiki/Perceptron)s and the connections between them.

Each **neuron** performs a simple computation: it takes multiple inputs, applies a weighted summation, and then passes the result through an activation function. Due to their simplicity, individual neurons can only solve **linear** problems. This mechanism is mathematically expressed as:

$$
y_{i} = f\left(\sum_{i} w_{i} x_{i} \right)
$$

where:
- $ y_i $ is the neuron's output,
- $ x_i $ are the input features,
- $ w_i $ are the weights assigned to each input,
- $ f $ is an activation function that introduces non-linearity.

### Learning Process

The key to neural networks is their ability to **learn** by adjusting weights based on errors. The simplest learning rule, used in the perceptron model, updates weights as follows:

$$
w_{i}(t+1) = w_{i}(t) + (d_{j} - y_{j}(t)) x_{j,i}
$$

where:
- $ d_j $ is the expected (true) output for the $ j $th input,
- $ y_j(t) $ is the predicted output,
- $ w_i(t) $ is the weight at time step $ t $.

Through repeated weight updates, the network gradually improves its accuracy in predicting outputs. However, a single-layer perceptron is still limited to solving linearly separable problems. To overcome this, we introduce **multi-layer networks**, which allow for more complex decision boundaries.

### **The XOR Problem & Non-Linearity Issue**
To illustrate the limitations of a single-layer perceptron, consider the XOR classification problem:

| $x_1$ | $x_2$ | Output |
|---|---|---|
| 0 | 0 | A |
| 0 | 1 | B |
| 1 | 0 | B |
| 1 | 1 | A |

In [ ]:
from sklearn.linear_model import Perceptron

In [ ]:
XOR_X, XOR_y = np.array([[0, 0], [0, 1], [1, 0], [1, 1]]), np.array([0, 1, 1, 0])
df = pd.DataFrame(data=XOR_X, columns=['x', 'y'])
df['label'] = XOR_y

In [ ]:
def plot_results_with_hyperplane(clf, clf_name, df, ax):
    x_min, x_max = df.x.min() - .5, df.x.max() + .5
    y_min, y_max = df.y.min() - .5, df.y.max() + .5

    xx, yy = np.meshgrid(np.arange(x_min, x_max, .02), np.arange(y_min, y_max, .02))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    ax.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired, shading='auto')
    ax.scatter(df.x, df.y, c=df.label, edgecolors='k')
    ax.set_title(clf_name)

In [ ]:
perceptron = Perceptron(verbose=2, random_state=42).fit(XOR_X, XOR_y)
perceptron

In [ ]:
fig, ax = plt.subplots()
plot_results_with_hyperplane(perceptron, 'perceptron', df, ax);

In [ ]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(XOR_y, perceptron.predict(XOR_X))
conf_mat

In [ ]:
sns.heatmap(conf_mat, annot=True, cbar=False)

#### Solving Non-Linear Problems

As we can see, a single neuron is not able to solve this non-linear problem. But they are not called **networks** for nothing! The power of artificial neural networks lies in their topology. If we connect more **neurons**, we get a (real) neural network. The neurons are organized into **layers**. The first layer is the **input layer**, followed by zero or more **hidden layer**(s), and finally, the **output layer**. Each layer can contain any number of neurons. Different topologies lead to different ANN subtypes.

<img src="pics/mlp.png" width=350 align="left"/>

The simplest form of ANN is the **Multi-Layer Perceptron (MLP)**, which consists of:
- An **input layer** receiving raw data
- One or more **hidden layers** that transform the data
- An **output layer** producing predictions

To allow for non-linearity, we use output (activation) functions such as:
- **Sigmoid**: $y(v_i) = \frac{1}{1 + e^{-v_i}}$ (good for probabilities)
- **Tanh**: $y(v_i) = \tanh(v_i)$ (better for zero-centered outputs)
- **ReLU**: $y(v_i) = \max(0, v_i)$ (common in deep networks)

The weight updating algorithm is called [**Backpropagation**](https://en.wikipedia.org/wiki/Backpropagation). It propagates the errors backward through the network, updating the weights of every neuron that contributed to the error. This follows the principles of [gradient descent](https://en.wikipedia.org/wiki/Backpropagation#Derivation), adjusting the weights based on the partial derivatives of the loss function.

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
mlp = MLPClassifier(random_state=42).fit(XOR_X, XOR_y)
mlp

In [ ]:
fig, ax = plt.subplots()
plot_results_with_hyperplane(mlp, 'mlp', df, ax)

In [ ]:
conf_mat = confusion_matrix(XOR_y, mlp.predict(XOR_X))
conf_mat

In [ ]:
sns.heatmap(conf_mat, annot=True, cbar=False)

A super nice tutorial can be found [here](https://github.com/rasbt/python-machine-learning-book/blob/master/code/ch12/ch12.ipynb), it is worth checking out.

##### Key Hyperparameters in MLPClassifier
- **`hidden_layer_sizes`**: Defines the number of neurons per hidden layer (e.g., `(4,)` means one hidden layer with 4 neurons)
- **`activation`**: Common choices include `"relu"`, `"tanh"`, `"logistic"` (sigmoid)
- **`solver`**: Optimization algorithm (`"adam"` is recommended for most cases)
- **`max_iter`**: Number of training iterations (increase if the model doesn’t converge)

This simple MLP demonstrates how adding hidden layers allows the network to learn **non-linear** patterns, solving problems that a single-layer perceptron cannot.

---